In [22]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
import cv2
import os
import pickle

In [24]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

root_dir = os.getcwd() + "\\data\\reorganized"
categories = os.listdir(root_dir)
width = 224
height = 224
dim = (width, height)
data = []


RuntimeError: Physical devices cannot be modified after being initialized

In [29]:
def make_data():
    for category in categories:
        path = os.path.join(root_dir, category)
        label = categories.index(category)

        for img_name in os.listdir(path):
            image_path = os.path.join(path, img_name)
            image = cv2.imread(image_path)

            try:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, dim)
                image = np.array(image, dtype=np.float32)
                data.append([image, label])
            except Exception as e:
                pass

    pik = open('pickle/ham10000classifier.pickle', 'wb')
    pickle.dump(data, pik)
    pik.close()

In [30]:
# make_data()

In [25]:
def load_data():
    pick = open('pickle/ham10000classifier.pickle', 'rb')
    data = pickle.load(pick)
    pick.close()

    feature = []
    labels = []

    for img, label in data:
        feature.append(img)
        labels.append(label)

    feature = np.array(feature, dtype=np.float32)
    labels = np.array(labels)
    feature = feature / 255.0
    return [feature, labels]

In [26]:
(feature, labels) = load_data()

In [27]:
acc_per_fold = []
loss_per_fold = []


In [28]:
num_folds = 3
kfold = KFold(n_splits=num_folds, random_state=3, shuffle=True)

In [29]:
lr = 0.0001
i = 1
j = 0
acc = []
rataloss = 0
rataacc = 0
rataprec = 0
ratarec = 0
rataf1 = 0
rata2loss = []
rata2acc = []
rata2prec = []
rata2rec = []
rata2f1 = []
#acclr=[]
#losslr=[]

akurasi_per_fold = []
prec_per_fold = []
rec_per_fold = []
f1_per_fold = []

a = []
b = []
learningrate = []
accscoremean = []
presscoremean = []
recscoremean = []
f1scoremean = []

In [30]:
from ModelResNet import create_model
from tensorflow.keras.optimizers import Adam
from matplotlib import pyplot as plt
from ConfusionMatrix import plot_confusion_matrix

model = create_model()
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=lr),
              metrics=['acc'])
inputs = feature
targets = labels
fold_no = 1

In [38]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

for train, test in kfold.split(inputs, targets):
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    r = model.fit(inputs[train[0:1]], targets[train[0:1]], validation_data=(inputs[test], targets[test]))

    scores = model.evaluate(inputs[test], targets[test], verbose=0)

    print(
        f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1] * 100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    Y_pred = model.predict(inputs[test], 30)
    y_pred = np.argmax(Y_pred, axis=1)
    plot_confusion_matrix(y_pred, targets[test], classes=categories, normalize=True, title='Confusion Matrix')
    #cm = confusion_matrix(y_pred,targets[test])
    accscore = accuracy_score(y_pred, targets[test])
    presscore = precision_score(y_pred, targets[test], average=None)
    recscore = recall_score(y_pred, targets[test], average=None)
    f1score = f1_score(y_pred, targets[test], average=None)

    akurasi_per_fold.append(np.mean(accscore))
    prec_per_fold.append(np.mean(presscore))
    rec_per_fold.append(np.mean(recscore))
    f1_per_fold.append(np.mean(f1score))

    accscore = 0
    presscore = 0
    recscore = 0
    f1score = 0
    plt.savefig('Kfold-' + str(i))
    plt.show()
    print(classification_report(targets[test], y_pred))

    fold_no += 1

print('rata-rata acc pada learning rate ', lr, ' adalah ', np.mean(akurasi_per_fold))
# print('rata-rata loss pada learning rate ', lr, 'adalah', losslr)
print('rata-rata presicion pada learning rate ', lr, 'adalah', np.mean(prec_per_fold))
print('rata-rata recall pada learning rate ', lr, 'adalah', np.mean(rec_per_fold))
print('rata-rata F1 Score pada learning rate ', lr, 'adalah', np.mean(f1_per_fold))

learningrate.append(lr)
lr = lr + 0.0111
i = i + 1
j = j + 1
fold_no = 1
print('=========================================================================')

------------------------------------------------------------------------
Training for fold 1 ...
1/1 [==============================] - ETA: 0s - loss: 1.6077 - acc: 1.0000

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.